In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import sys
from itertools import product
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from lifelines import KaplanMeierFitter
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

In [ ]:
src_path: str = "../../src"
sys.path.append(src_path)

In [ ]:
from components.functional_analysis.orgdb import OrgDB
from data.utils import gene_expression_levels
from r_wrappers.utils import map_gene_id

org_db = OrgDB("Homo sapiens")

In [ ]:
tcga_su2c_root = Path("/media/ssd/Perez/storage/TCGA_PRAD_SU2C_RNASeq")
tcga_survival_plots = tcga_su2c_root.joinpath("tcga_survival_plots")
tcga_survival_plots.mkdir(exist_ok=True, parents=True)
su2c_survival_plots = tcga_su2c_root.joinpath("su2c_survival_plots")
su2c_survival_plots.mkdir(exist_ok=True, parents=True)
su2c_root = Path("/media/ssd/Perez/storage/SU2C_PCF_2019_RNASeq")

## Survival analysis for potential biomarkers

---


### 1. Load counts and annotation data


In [ ]:
counts_vst = pd.read_csv(
    tcga_su2c_root.joinpath(
        "deseq2/sample_cluster_no_replicates_MET_BB+NORM+PRIM__vst.csv"
    ),
    index_col=0,
)
counts_vst.index = map_gene_id(counts_vst.index, org_db, "ENSEMBL", "SYMBOL")
counts_vst = counts_vst[~counts_vst.index.str.contains("/", na=True)]

In [ ]:
annot_df = pd.read_csv(
    tcga_su2c_root.joinpath("data/samples_annotation_tcga_prad_su2c_clusters.csv"),
    index_col=0,
)
annot_df = annot_df[
    annot_df["sample_cluster_no_replicates"].isin(("NORM", "PRIM", "MET_BB"))
]
annot_df = annot_df.sort_values(
    "sample_cluster_no_replicates", key=lambda x: x.str.len()
)

In [ ]:
tcga_clinical = pd.read_csv(
    tcga_su2c_root.joinpath("data").joinpath("clinical_data.csv"), index_col=2
)
tcga_clinical = tcga_clinical.loc[tcga_clinical.index.intersection(annot_df.index), :]
tcga_clinical = tcga_clinical[tcga_clinical["definition"] == "Primary solid Tumor"]

su2c_clinical = pd.read_csv(
    su2c_root.joinpath("samples_annotations").joinpath(
        "samples_annotation_rna_downloaded.csv"
    ),
    index_col=0,
)
su2c_clinical = su2c_clinical.loc[su2c_clinical.index.intersection(annot_df.index), :]

### 2. Load potential biomarkers


In [ ]:
from itertools import chain

with tcga_su2c_root.joinpath("intersecting_ml_wgcna_genes.json").open("r") as fp:
    all_biomarkers_dict = json.load(fp)

all_biomarkers = list(chain(*all_biomarkers_dict.values()))
print(all_biomarkers)

In [ ]:
tcga_su2c_root.joinpath("intersecting_ml_wgcna_genes.tsv").write_text(
    "\n".join(all_biomarkers)
)

### 3. Contrast gene expression of potential biomarkers with clinical data


In [ ]:
for gene, percentile in product(all_biomarkers, (10, 20, 30)):
    tcga_clinical[gene] = counts_vst.loc[gene, tcga_clinical.index]
    tcga_clinical = gene_expression_levels(
        expr_df=tcga_clinical,
        gene_expr_col=gene,
        gene_expr_level=f"{gene}_levels_{percentile}",
        percentile=percentile,
    )
    su2c_clinical[gene] = counts_vst.loc[gene, su2c_clinical.index]
    su2c_clinical = gene_expression_levels(
        expr_df=su2c_clinical,
        gene_expr_col=gene,
        gene_expr_level=f"{gene}_levels_{percentile}",
        percentile=percentile,
    )

In [ ]:
tcga_clinical.to_csv(tcga_su2c_root.joinpath("tcga_clinical_biomarkers.csv"))
su2c_clinical.to_csv(tcga_su2c_root.joinpath("su2c_clinical_biomarkers.csv"))

#### 3.1. TCGA


##### 3.1.1. Survival T-test between gene expression distributions


In [ ]:
tcga_prad_status_tests = pd.DataFrame(
    index=all_biomarkers, columns=["t_statistic", "p_value"]
)

for gene in all_biomarkers:
    g1 = tcga_clinical[
        (tcga_clinical["vital_status"] == "Alive")
        & (tcga_clinical["definition"] == "Primary solid Tumor")
    ][gene]
    g2 = tcga_clinical[
        (tcga_clinical["vital_status"] == "Dead")
        & (tcga_clinical["definition"] == "Primary solid Tumor")
    ][gene]

    tcga_prad_status_tests.loc[gene, :] = ttest_ind(g1, g2)

tcga_prad_status_tests["p_adj"] = multipletests(
    tcga_prad_status_tests["p_value"], method="fdr_bh"
)[1]
tcga_prad_status_tests = tcga_prad_status_tests.sort_values(["p_adj", "p_value"])
tcga_prad_status_tests.to_csv(tcga_su2c_root.joinpath("tcga_prad_status_tests.csv"))
tcga_prad_status_tests

##### 3.1.2. Kaplan-Meier plot


In [ ]:
for gene, percentile in product(all_biomarkers, (10, 20, 30)):
    try:
        T = tcga_clinical["days_to_collection"] * 30
        E = tcga_clinical["vital_status"] == "Dead"
        indx_low = (tcga_clinical[f"{gene}_levels_{percentile}"] == "low") & (~T.isna())
        indx_high = (tcga_clinical[f"{gene}_levels_{percentile}"] == "high") & (
            ~T.isna()
        )

        fig = plt.figure(figsize=(7, 7))
        ax = fig.add_subplot(111)

        kmf = KaplanMeierFitter()

        kmf.fit(
            T[indx_low],
            event_observed=E[indx_low],
            label=f"Low gene expression (n={sum(indx_low)})",
        )
        kmf.plot_survival_function(ax=ax)

        kmf.fit(
            T[indx_high],
            event_observed=E[indx_high],
            label=f"High gene expression (n={sum(indx_high)})",
        )
        kmf.plot_survival_function(ax=ax, at_risk_counts=True)
        plt.title(f"TCGA-PRAD. Survival based in {gene} expression level.")

        plt.tight_layout()

        plt.savefig(tcga_survival_plots.joinpath(f"{gene}_{percentile}.pdf"))
        plt.close()
    except Exception:
        continue

#### 3.2. SU2C-PCF


In [ ]:
su2c_pcf_status_tests = pd.DataFrame(
    index=all_biomarkers, columns=["t_statistic", "p_value"]
)

for gene in all_biomarkers:
    g1 = su2c_clinical[su2c_clinical["OS_STATUS"] == "1:DECEASED"][gene]
    g2 = su2c_clinical[su2c_clinical["OS_STATUS"] == "0:LIVING"][gene]

    su2c_pcf_status_tests.loc[gene, :] = ttest_ind(g1, g2)

su2c_pcf_status_tests["p_adj"] = multipletests(
    su2c_pcf_status_tests["p_value"], method="fdr_bh"
)[1]
su2c_pcf_status_tests = su2c_pcf_status_tests.sort_values(["p_adj", "p_value"])
su2c_pcf_status_tests.to_csv(tcga_su2c_root.joinpath("su2c_pcf_status_tests.csv"))
su2c_pcf_status_tests

##### 3.2.2. Kaplan-Meier plot


In [ ]:
for gene, percentile in product(all_biomarkers, (10, 20, 30)):
    try:
        T = (
            su2c_clinical["AGE_AT_PROCUREMENT"].astype(float)
            - su2c_clinical["AGE_AT_DIAGNOSIS"].astype(float)
        ) * 30
        E = su2c_clinical["OS_STATUS"] == "1:DECEASED"
        indx_low = (su2c_clinical[f"{gene}_levels_{percentile}"] == "low") & (~T.isna())
        indx_high = (su2c_clinical[f"{gene}_levels_{percentile}"] == "high") & (
            ~T.isna()
        )

        fig = plt.figure(figsize=(7, 7))
        ax = fig.add_subplot(111)

        kmf = KaplanMeierFitter()

        kmf.fit(
            T[indx_low],
            event_observed=E[indx_low],
            label=f"Low gene expression (n={sum(indx_low)})",
        )
        kmf.plot_survival_function(ax=ax)

        kmf.fit(
            T[indx_high],
            event_observed=E[indx_high],
            label=f"High gene expression (n={sum(indx_high)})",
        )
        kmf.plot_survival_function(ax=ax, at_risk_counts=True)
        plt.title(f"SU2C-PCF. Survival based in {gene} expression level.")

        plt.tight_layout()

        plt.savefig(su2c_survival_plots.joinpath(f"{gene}_{percentile}.pdf"))
        plt.close()
    except Exception:
        continue

---
